# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [2]:
import os
import glob
import psycopg2
import pandas as pd
import json
from sql_queries import *

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
filepath = '/home/workspace/data/song_data/'


In [6]:
song_files = get_files(filepath)





In [7]:
for single_file in song_files:
    with open(single_file) as single_song_data:
        print("single_file",single_file)
       

single_file /home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json
single_file /home/workspace/data/song_data/A/A/A/TRAAAVG12903CFA543.json
single_file /home/workspace/data/song_data/A/A/A/TRAAARJ128F9320760.json
single_file /home/workspace/data/song_data/A/A/A/TRAAAPK128E0786D96.json
single_file /home/workspace/data/song_data/A/A/A/TRAAABD128F429CF47.json
single_file /home/workspace/data/song_data/A/A/A/TRAAAMO128F1481E7F.json
single_file /home/workspace/data/song_data/A/A/A/TRAAAMQ128F1460CD3.json
single_file /home/workspace/data/song_data/A/A/A/TRAAAVO128F93133D4.json
single_file /home/workspace/data/song_data/A/A/A/TRAAAEF128F4273421.json
single_file /home/workspace/data/song_data/A/A/A/TRAAAFD128F92F423A.json
single_file /home/workspace/data/song_data/A/A/A/TRAAADZ128F9348C2E.json
single_file /home/workspace/data/song_data/A/A/A/.ipynb_checkpoints/TRAAABD128F429CF47-checkpoint.json
single_file /home/workspace/data/song_data/A/A/A/.ipynb_checkpoints/TRAAAAW128F429D538-checkpo

In [8]:
df = pd.read_json(single_file,typ='series')
print(df.head)


<bound method NDFrame.head of num_songs                            1
artist_id           AREVWGE1187B9B890A
artist_latitude                -13.442
artist_longitude              -41.9952
artist_location              Noci (BA)
artist_name                 Bitter End
song_id             SOFCHDR12AB01866EF
title                      Living Hell
duration                       282.435
year                                 0
dtype: object>


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = df[["song_id","title","artist_id","year","duration"]]
print(song_data)

song_id      SOFCHDR12AB01866EF
title               Living Hell
artist_id    AREVWGE1187B9B890A
year                          0
duration                282.435
dtype: object


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]
print(artist_data)

artist_id      AREVWGE1187B9B890A
artist_name            Bitter End
location                      NaN
lattitude                     NaN
longitude                     NaN
dtype: object


/opt/conda/lib/python3.6/site-packages/pandas/core/series.py:851: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [28]:
filepath = '/home/workspace/data/log_data/'


In [29]:
log_files = get_files(filepath)
print(log_files)

['/home/workspace/data/log_data/2018/11/2018-11-30-events.json', '/home/workspace/data/log_data/2018/11/2018-11-11-events.json', '/home/workspace/data/log_data/2018/11/2018-11-23-events.json', '/home/workspace/data/log_data/2018/11/2018-11-24-events.json', '/home/workspace/data/log_data/2018/11/2018-11-25-events.json', '/home/workspace/data/log_data/2018/11/2018-11-20-events.json', '/home/workspace/data/log_data/2018/11/2018-11-15-events.json', '/home/workspace/data/log_data/2018/11/2018-11-01-events.json', '/home/workspace/data/log_data/2018/11/2018-11-03-events.json', '/home/workspace/data/log_data/2018/11/2018-11-13-events.json', '/home/workspace/data/log_data/2018/11/2018-11-10-events.json', '/home/workspace/data/log_data/2018/11/2018-11-28-events.json', '/home/workspace/data/log_data/2018/11/2018-11-26-events.json', '/home/workspace/data/log_data/2018/11/2018-11-04-events.json', '/home/workspace/data/log_data/2018/11/2018-11-14-events.json', '/home/workspace/data/log_data/2018/11/

In [30]:
for single_logfile in log_files:
    with open(single_logfile) as log_data:
        print("single_logfile",single_logfile)

single_logfile /home/workspace/data/log_data/2018/11/2018-11-30-events.json
single_logfile /home/workspace/data/log_data/2018/11/2018-11-11-events.json
single_logfile /home/workspace/data/log_data/2018/11/2018-11-23-events.json
single_logfile /home/workspace/data/log_data/2018/11/2018-11-24-events.json
single_logfile /home/workspace/data/log_data/2018/11/2018-11-25-events.json
single_logfile /home/workspace/data/log_data/2018/11/2018-11-20-events.json
single_logfile /home/workspace/data/log_data/2018/11/2018-11-15-events.json
single_logfile /home/workspace/data/log_data/2018/11/2018-11-01-events.json
single_logfile /home/workspace/data/log_data/2018/11/2018-11-03-events.json
single_logfile /home/workspace/data/log_data/2018/11/2018-11-13-events.json
single_logfile /home/workspace/data/log_data/2018/11/2018-11-10-events.json
single_logfile /home/workspace/data/log_data/2018/11/2018-11-28-events.json
single_logfile /home/workspace/data/log_data/2018/11/2018-11-26-events.json
single_logfi

In [31]:
df = pd.read_json(single_logfile,lines=True)
df.head()
df['ts'] = pd.to_datetime(df['ts'], unit='ms')
print(df.loc[1])

artist                                              Randy Crawford
auth                                                     Logged In
firstName                                                    Chloe
gender                                                           F
itemInSession                                                   54
lastName                                                    Cuevas
length                                                     251.402
level                                                         paid
location                         San Francisco-Oakland-Hayward, CA
method                                                         PUT
page                                                      NextSong
registration                                           1.54094e+12
sessionId                                                      648
song                           Rio De Janeiro Blue (Album Version)
status                                                        

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [32]:
df_filtered_records = df[(df.page=="NextSong")]
print(df_filtered_records.loc[1])


artist                                              Randy Crawford
auth                                                     Logged In
firstName                                                    Chloe
gender                                                           F
itemInSession                                                   54
lastName                                                    Cuevas
length                                                     251.402
level                                                         paid
location                         San Francisco-Oakland-Hayward, CA
method                                                         PUT
page                                                      NextSong
registration                                           1.54094e+12
sessionId                                                      648
song                           Rio De Janeiro Blue (Album Version)
status                                                        

In [33]:
t = df_filtered_records.ts
t.head()

0   2018-11-17 00:02:24.796
1   2018-11-17 00:06:40.796
2   2018-11-17 00:10:51.796
3   2018-11-17 00:14:35.796
4   2018-11-17 00:17:39.796
Name: ts, dtype: datetime64[ns]

In [34]:
timestamp1=pd.to_datetime(df_filtered_records['ts'],unit='ms',errors='coerce')
timestamp= timestamp1.to_frame()

month1=pd.to_datetime(df_filtered_records['ts'],unit='ms',errors='coerce').dt.month
month= month1.to_frame()

year1=pd.to_datetime(df_filtered_records['ts'],unit='ms',errors='coerce').dt.year
year= year1.to_frame()

hour1=pd.to_datetime(df_filtered_records['ts'],unit='ms',errors='coerce').dt.hour
hour= hour1.to_frame()

day1=pd.to_datetime(df_filtered_records['ts'],unit='ms',errors='coerce').dt.day
day= day1.to_frame()

weekday1=pd.to_datetime(df_filtered_records['ts'],unit='ms',errors='coerce').dt.weekday_name
weekday= weekday1.to_frame()

dayofweek1=pd.to_datetime(df_filtered_records['ts'],unit='ms',errors='coerce').dt.dayofweek
dayofweek= dayofweek1.to_frame()

week1=pd.to_datetime(df_filtered_records['ts'],unit='ms',errors='coerce').dt.week
week= week1.to_frame()

In [35]:
time_df=pd.DataFrame(columns=['timestamp','hour','day','week','month,''year','weekday'])
time_df= pd.concat([timestamp1,hour1,day1,week1,month1,year1,dayofweek1],axis=1)
time_df.columns=['starttime','hour','day','week','month','year','weekday']
time_df.head()



,starttime,hour,day,week,month,year,weekday
0,2018-11-17 00:02:24.796,0,17,46,11,2018,5
1,2018-11-17 00:06:40.796,0,17,46,11,2018,5
2,2018-11-17 00:10:51.796,0,17,46,11,2018,5
3,2018-11-17 00:14:35.796,0,17,46,11,2018,5
4,2018-11-17 00:17:39.796,0,17,46,11,2018,5


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [36]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [37]:
user_df = df_filtered_records[['userId','firstName','lastName','gender','level']]
print(user_df)

    userId firstName   lastName gender level
0       49     Chloe     Cuevas      F  paid
1       49     Chloe     Cuevas      F  paid
2       49     Chloe     Cuevas      F  paid
3       49     Chloe     Cuevas      F  paid
4       49     Chloe     Cuevas      F  paid
5       49     Chloe     Cuevas      F  paid
6       49     Chloe     Cuevas      F  paid
7       88  Mohammad  Rodriguez      M  free
8       49     Chloe     Cuevas      F  paid
9       49     Chloe     Cuevas      F  paid
10      49     Chloe     Cuevas      F  paid
11      49     Chloe     Cuevas      F  paid
12      49     Chloe     Cuevas      F  paid
13      49     Chloe     Cuevas      F  paid
14      49     Chloe     Cuevas      F  paid
15      49     Chloe     Cuevas      F  paid
16      49     Chloe     Cuevas      F  paid
17      49     Chloe     Cuevas      F  paid
18      49     Chloe     Cuevas      F  paid
19      49     Chloe     Cuevas      F  paid
20      49     Chloe     Cuevas      F  paid
21      49

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [38]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and S ongplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [40]:
for index, row in df_filtered_records.iterrows():

    # get songid and artistid from song and artist tables

    cur.execute(song_select, (row.song, row.artist,row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
        
    print(results)
    # insert songplay record
    songplay_data = (row.ts,row.userId,row.level,songid,artistid,row.sessionId,row.location,row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

None


IndexError: tuple index out of range

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [24]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.